In [1]:
from transformers import ElectraTokenizer, ElectraForSequenceClassification
import torch

# Load the model and tokenizer
model_name = 'gooohjy/suicidal-electra'
model = ElectraForSequenceClassification.from_pretrained(model_name)
tokenizer = ElectraTokenizer.from_pretrained(model_name)

# Specify device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Move model to device
model.to(device)

def get_label_probabilities(text):
    # Tokenize the text
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)

    # Move inputs to device
    inputs = {name: tensor.to(device) for name, tensor in inputs.items()}
    
    # Get the model outputs
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Get the probabilities with softmax
    probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
    
    # Move probs back to CPU if necessary for further processing
    probs = probs.cpu().numpy()[0]
    
    return probs


/home/kororu/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Example usage:
text = "It is such a wonderful day to kill myself and bring me a peace."
probs = get_label_probabilities(text)
print(f'suicide: {probs[1]:.5f}, non_suicide: {probs[0]:.5f}')

suicide: 0.99925, non_suicide: 0.00075


In [2]:
import twitter_sentiment
import kaggle_model_2

# Initialize the models
twitter_sentiment.initialize_model()
kaggle_model_2.initialize_model()

# Test text
text = """I want to be erased from everyone's memory
I want to fall asleep and not wake up
I want to be murderd so I'm not to blame 
I just want to escape
I cut every night because my step dad
Fuck him"""

# Get label probabilities
twitter_score = twitter_sentiment.get_label_probabilities(text)
kaggle_score = kaggle_model_2.get_label_probabilities(text)

# Print the scores
print(f"Twitter Sentiment Score: {twitter_score}")
print(f"Kaggle Model 2 Score: {kaggle_score}")


Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Twitter Sentiment Score: 0.9901023507118225
Kaggle Model 2 Score: 0.9518818259239197


In [3]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification
import torch

model_name = 'rafalposwiata/deproberta-large-depression'
model = None
tokenizer = None
device = None

def initialize_model():
    global model, tokenizer, device
    model = RobertaForSequenceClassification.from_pretrained(model_name)
    tokenizer = RobertaTokenizer.from_pretrained(model_name)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

def get_label_probabilities(text):
    if model is None or tokenizer is None:
        initialize_model()
    
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
    inputs = {name: tensor.to(device) for name, tensor in inputs.items()}
    
    with torch.no_grad():
        outputs = model(**inputs)
    
    probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
    probs = probs.cpu().numpy()[0]
    
    # Returning the probability of label 1 (suicide)
    return [probs[0], probs[1], probs[2]]  # [label_0_score, label_1_score]


In [7]:
# Initialize the models
initialize_model()

# Test text
text = """Well, it is time to the end. I QUIT"""

# Get label probabilities
twitter_score = get_label_probabilities(text)

# Print the scores
print(f"Twitter Sentiment Score: {twitter_score[0]}")

Twitter Sentiment Score: 0.08310867100954056


In [ ]:
# severe, moderate, not severe